In [5]:
import json
import pandas as pd
from datetime import datetime
from TAI.data import DataMaster

dm = DataMaster()

df = dm.load_local(data_folder='', 
                    file_name = 'sp500_ohlc_data.parquet',
                    use_polars = False,
                    load_all = False, 
                    selected_files = ['sp500_ohlc_data.parquet'])[:50]
df_dict = df.to_dict(orient='records')

def convert_to_json(data):
    def convert_timestamp(item):
        if isinstance(item, dict):
            # Convert 'Timestamp' to ISO 8601 format
            if 'timestamp' in item and isinstance(item['timestamp'], pd.Timestamp):
                item['timestamp'] = item['timestamp'].isoformat()
        return item

    # Apply the conversion to each dictionary in the list
    cleaned_data = [convert_timestamp(entry) for entry in data]
    
    # Convert to JSON format (as Python object, not a string)
    return cleaned_data
df_dict


File loaded from sp500_ohlc_data.parquet


[{'symbol': 'A',
  'timestamp': Timestamp('2016-01-04 05:00:00+0000', tz='UTC'),
  'open': 38.36,
  'high': 38.48,
  'low': 37.69,
  'close': 38.01,
  'volume': 3678650.0},
 {'symbol': 'A',
  'timestamp': Timestamp('2016-01-05 05:00:00+0000', tz='UTC'),
  'open': 38.05,
  'high': 38.26,
  'low': 37.69,
  'close': 37.88,
  'volume': 2800925.0},
 {'symbol': 'A',
  'timestamp': Timestamp('2016-01-06 05:00:00+0000', tz='UTC'),
  'open': 37.59,
  'high': 38.3,
  'low': 37.42,
  'close': 38.05,
  'volume': 2289681.0},
 {'symbol': 'A',
  'timestamp': Timestamp('2016-01-07 05:00:00+0000', tz='UTC'),
  'open': 37.5,
  'high': 37.51,
  'low': 36.26,
  'close': 36.44,
  'volume': 3762400.0},
 {'symbol': 'A',
  'timestamp': Timestamp('2016-01-08 05:00:00+0000', tz='UTC'),
  'open': 36.64,
  'high': 37.09,
  'low': 35.94,
  'close': 36.05,
  'volume': 4449196.0},
 {'symbol': 'A',
  'timestamp': Timestamp('2016-01-11 05:00:00+0000', tz='UTC'),
  'open': 36.17,
  'high': 36.34,
  'low': 34.95,
  'clo

In [6]:
df_json = convert_to_json(df_dict)
df_json

[{'symbol': 'A',
  'timestamp': '2016-01-04T05:00:00+00:00',
  'open': 38.36,
  'high': 38.48,
  'low': 37.69,
  'close': 38.01,
  'volume': 3678650.0},
 {'symbol': 'A',
  'timestamp': '2016-01-05T05:00:00+00:00',
  'open': 38.05,
  'high': 38.26,
  'low': 37.69,
  'close': 37.88,
  'volume': 2800925.0},
 {'symbol': 'A',
  'timestamp': '2016-01-06T05:00:00+00:00',
  'open': 37.59,
  'high': 38.3,
  'low': 37.42,
  'close': 38.05,
  'volume': 2289681.0},
 {'symbol': 'A',
  'timestamp': '2016-01-07T05:00:00+00:00',
  'open': 37.5,
  'high': 37.51,
  'low': 36.26,
  'close': 36.44,
  'volume': 3762400.0},
 {'symbol': 'A',
  'timestamp': '2016-01-08T05:00:00+00:00',
  'open': 36.64,
  'high': 37.09,
  'low': 35.94,
  'close': 36.05,
  'volume': 4449196.0},
 {'symbol': 'A',
  'timestamp': '2016-01-11T05:00:00+00:00',
  'open': 36.17,
  'high': 36.34,
  'low': 34.95,
  'close': 35.45,
  'volume': 2951588.0},
 {'symbol': 'A',
  'timestamp': '2016-01-12T05:00:00+00:00',
  'open': 35.9,
  'high

In [ ]:
from TAI.source import Alpaca
import pandas as pd
import datetime

def get_sp500_symbols():
    """
    Fetches the list of S&P 500 company symbols from Wikipedia.
    """
    # URL of the Wikipedia page containing the S&P 500 companies
    sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    
    # Read the tables on the Wikipedia page
    sp500_table = pd.read_html(sp500_url)
    
    # The first table contains the list of companies
    sp500_df = sp500_table[0]
    
    # Extract the 'Symbol' column as a list
    symbols = sp500_df['Symbol'].tolist()
    
    # Replace any periods in the symbols with hyphens (if required by the API)
    symbols = [symbol.replace('.', '-') for symbol in symbols]
    
    return symbols

get_sp500_symbols()

In [ ]:

alpaca = Alpaca()
lookback_period = 3652  # Approximate number of days in 10 years (including leap years)
symbols =  get_sp500_symbols()

# Fetch the historical data
df = alpaca.get_stock_historical_sync(
        symbol_or_symbols=symbols,
        lookback_period=lookback_period,
        timeframe='Day',
        ohlc=True  # Get standard OHLC data
        )

# Save the DataFrame to a Parquet file
# df.to_parquet('sp500_10years.parquet')
df

In [ ]:
from TAI.source import Alpaca
ap = Alpaca()

In [ ]:
ap.get_stock_historical(symbol_or_symbols=['SPY','MSFT'], 
                             lookback_period=3650,
                             timeframe='Day', 
                             end=None, currency='USD',
                             limit=None, adjustment='all', feed=None,
                             asof=None, sort='asc', raw=False, ohlc=True)

In [ ]:
def get_sp500_symbols():
    """
    Fetches the list of S&P 500 company symbols using the IEX Cloud API.
    """
    import os
    IEX_CLOUD_API_KEY = os.getenv('IEX_CLOUD_API_KEY')
    if not IEX_CLOUD_API_KEY:
        raise Exception("Please set your IEX Cloud API key in the IEX_CLOUD_API_KEY environment variable.")

    iex_api_url = f'https://cloud.iexapis.com/stable/ref-data/symbols'
    response = requests.get(iex_api_url)

    if response.status_code == 200:
        data = response.json()
        # Filter for S&P 500 companies
        symbols = [item['symbol'] for item in data if 'S&P 500' in item['tags']]
        return symbols
    else:
        raise Exception(f"Failed to fetch symbols from IEX Cloud. Status code: {response.status_code}")
get_sp500_symbols()